In [1]:
import os
Dataset='Dataset'
Data_Dir=os.listdir(Dataset)
print(Data_Dir)

['without_mask', 'with_mask']


In [2]:
# Import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

img_rows, img_cols = 112, 112

images = []
labels = []

for category in Data_Dir:
    folder_path = os.path.join(Dataset, category)
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        img=cv2.imread(img_path)

        try:
            #Coverting the image into gray scale
            grayscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            #resizing the gray scaled image into size 56x56 in order to keep size of the images consistent           
            resized_img=cv2.resize(grayscale_img,(img_rows, img_cols))
            images.append(resized_img)
            labels.append(category)
        # Exception Handling in case any error occurs
        except Exception as e:
            print('Exception:',e)
        
images=np.array(images)/255.0
images=np.reshape(images,(images.shape[0],img_rows, img_cols,1))
# Perform one hot encoding on the labels since the label are in textual form
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
labels = np.array(labels)

(train_X, test_X, train_y, test_y) = train_test_split(images, labels, test_size=0.25,random_state=0)

In [3]:
# Build Covolutional Neural Network (CNN) Classification Model
# Import Necessary Keras Libraries
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D

# Define model paramters
num_classes = 2
batch_size = 32

# Build CNN model using Sequential API
model=Sequential()

#First layer group containing Convolution, Relu and MaxPooling layers
model.add(Conv2D(64,(3,3),input_shape=(img_rows, img_cols, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Second layer group containing Convolution, Relu and MaxPooling layers
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatten and Dropout Layer to stack the output convolutions above as well as cater overfitting
model.add(Flatten())
model.add(Dropout(0.5))

# Softmax Classifier
model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 64)      640       
_________________________________________________________________
activation (Activation)      (None, 110, 110, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 53, 53, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 86528)             0

In [4]:
# Plot the model
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='face_mask_detection_architecture.png')

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [5]:
# Train the Model
from keras.optimizers import Adam
epochs = 50

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

fitted_model = model.fit(
    train_X,
    train_y,
    epochs = epochs,
    validation_split=0.25)

Epoch 1/50
25/25 [==============================] - 18s 716ms/step - loss: 1.1682 - accuracy: 0.5060 - val_loss: 0.7148 - val_accuracy: 0.5271
Epoch 2/50
25/25 [==============================] - 21s 839ms/step - loss: 0.6759 - accuracy: 0.5460 - val_loss: 0.5417 - val_accuracy: 0.7171
Epoch 3/50
25/25 [==============================] - 20s 797ms/step - loss: 0.5127 - accuracy: 0.7481 - val_loss: 0.3840 - val_accuracy: 0.8217
Epoch 4/50
25/25 [==============================] - 21s 837ms/step - loss: 0.3893 - accuracy: 0.8214 - val_loss: 0.3948 - val_accuracy: 0.8023
Epoch 5/50
25/25 [==============================] - 21s 835ms/step - loss: 0.3084 - accuracy: 0.8660 - val_loss: 0.2867 - val_accuracy: 0.8992
Epoch 6/50
25/25 [==============================] - 20s 818ms/step - loss: 0.2246 - accuracy: 0.9134 - val_loss: 0.2500 - val_accuracy: 0.8798
Epoch 7/50
25/25 [==============================] - 21s 853ms/step - loss: 0.2229 - accuracy: 0.9211 - val_loss: 0.2184 - val_accuracy: 0.9109

In [6]:
# Plot the training loss and accuracy
from matplotlib import pyplot as plt
# Plot training and validation loss
plt.plot(fitted_model.history['loss'], 'r', label='training loss')
plt.plot(fitted_model.history['val_loss'], label='validation loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss Value')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(fitted_model.history['accuracy'], 'r', label='training accuracy')
plt.plot(fitted_model.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy Value')
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [9]:
# save or serialize the model
model.save("mask_detector.model", save_format="h5")

In [6]:
#ALARM CODE
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

from pygame import mixer
mixer.init()
sound = mixer.Sound('alarm.wav')
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # initialize our list of faces, their corresponding locations, and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all* faces at the same time rather than one-by-one predictions in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding locations
    return (locs, preds)

# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("mask_detector.model")

# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it  to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        if(label =="No Mask"):
            sound.play()
            print("Beep")

        elif(label=="Mask"):
            sound.stop()
            print("No Beep") 

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output frame
        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == 27:
        cv2.destroyAllWindows()
        vs.stop()
        break

pygame 2.0.1 (SDL 2.0.14, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
[INFO] starting video stream...
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
No Beep
(1, 1

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
No Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
(1, 1, 200, 7)
Beep
